# Import Libraries

In [ ]:
# !pip install seaborn

In [1]:
import numpy as np
import pandas as pd
from   osgeo import gdal
from   sklearn.preprocessing import MinMaxScaler
from   sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import skimage.feature
import skimage.filters 
import skimage.morphology
from   skimage import io
import skimage.measure
from   skimage.feature import graycomatrix, graycoprops

In [2]:

import time
import matplotlib.pyplot as plt
from  sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import squareform
from skimage import exposure


# Read Image

In [35]:
image_path = (r'IMG.tif')
dataset = gdal.Open(image_path)
image = dataset.ReadAsArray()
gt = dataset.GetGeoTransform()

X_size = gt[1]
Y_size = -gt[5]
width = dataset.RasterXSize
height = dataset.RasterYSize
num_bands = dataset.RasterCount

print(gt)
print ('------------')
print ('Number of Bands', num_bands)
print ('width', width)
print('height', height)
print(X_size)
print (Y_size)
print('--------------')


(-367700.0, 0.5, 0.0, -1187921.0, 0.0, -0.5)
------------
Number of Bands 18
width 16836
height 8679
0.5
0.5
--------------


In [36]:
image.shape

(18, 8679, 16836)

# Band Statistics

In [37]:
num_bands, height, width = image.shape

for band in range(num_bands):
    min_value = np.min(image[band, :, :])
    max_value = np.max(image[band, :, :])
    print(f"Band {band + 1}: Min = {min_value}, Max  = {max_value}")

Band 1: Min = 0.0, Max  = 1.0
Band 2: Min = 0.0, Max  = 1.0
Band 3: Min = 0.0, Max  = 1.0
Band 4: Min = 0.0, Max  = 1.0
Band 5: Min = 0.0, Max  = 1.0
Band 6: Min = 0.0, Max  = 1.0
Band 7: Min = 0.0, Max  = 1.0
Band 8: Min = 0.0, Max  = 1.0
Band 9: Min = 0.0, Max  = 1.0
Band 10: Min = 0.0, Max  = 1.0
Band 11: Min = 0.0, Max  = 1.0
Band 12: Min = 0.0, Max  = 1.0
Band 13: Min = 0.0, Max  = 1.0
Band 14: Min = 0.0, Max  = 1.0
Band 15: Min = 0.0, Max  = 1.0
Band 16: Min = 0.0, Max  = 1.0
Band 17: Min = 0.0, Max  = 1.0
Band 18: Min = 0.0, Max  = 1.0


# Read Samples

In [39]:
samples = pd.read_csv(r'sample.csv')
samples.dropna(inplace = True)
print (samples.shape)
class_count = samples['Value'].value_counts()
print(class_count)
samples.head(5)

(1019, 26)
tat
7    371
4    339
8    309
Name: count, dtype: int64


,rand_point_id,OBJECTID,Id,tat,area,kontrola,Shape_Leng,Shape_Area,B1,B2,...,B9,B10,B11,B12,B13,B14,B15,B16,B17,B18
0,0,197,0,4,74.764935,0,35.935776,74.7649,0.627451,0.689732,...,0.309398,0.812497,0.578740,0.721569,0.145098,0.645669,0.610236,0.686275,0.024038,0.076142
1,1,197,0,4,74.764935,0,35.935776,74.7649,0.780392,0.810268,...,0.377322,0.003662,0.693548,0.854902,0.184000,0.780877,0.766129,0.796078,0.040909,0.023529
4,4,197,0,4,74.764935,0,35.935776,74.7649,0.572549,0.600446,...,0.348424,0.499886,0.543307,0.670588,0.129412,0.598425,0.562992,0.635294,0.014423,0.081218
5,5,197,0,4,74.764935,0,35.935776,74.7649,0.701961,0.707589,...,0.531724,0.968811,0.551181,0.913725,0.364706,0.688976,0.598425,0.729412,0.129808,0.035533
6,6,197,0,4,74.764935,0,35.935776,74.7649,0.533333,0.569196,...,0.068820,0.123049,0.488189,0.564706,0.078431,0.523622,0.496063,0.541176,0.048077,0.010152


In [40]:

f_name  = [
      "B1","B2", "B3", "B4", "B5", "B6", "B7", 
        "B8", "B9","B10", "B11", "B12", "B13",
          "B14","B15", "B16", "B17", "B18"
]


label = "value"

X = samples[f_name]
y = samples[label]


df = pd.concat([X, y], axis=1)
df.dropna(inplace=True)

# Separate the feature matrix X and the target variable y
X = df[f_name]
y = df[label]


# Train the Model(Random Forest)

In [47]:
X_train , X_test ,y_train ,y_test = train_test_split(X ,y , test_size=0.2,
                                                    random_state= 42)

model = RandomForestClassifier(random_state= 42)
model.fit(X_train , y_train)

RandomForestClassifier(random_state=42)

In [48]:
importances = model.feature_importances_

# Sort the importances in descending order
sorted_indices = sorted(range(len(importances)), key=lambda k: importances[k], reverse=True)
sorted_features = [f_name[i] for i in sorted_indices]
sorted_importances = [importances[i] for i in sorted_indices]

# Print the sorted feature importances
for feature, importance in zip(sorted_features, sorted_importances):
    print(f" {feature:<26} {round(importance,3)}")


 B11                        0.202
 B15                        0.159
 B2                         0.123
 B14                        0.114
 B3                         0.101
 B16                        0.067
 B1                         0.058
 B13                        0.031
 B4                         0.021
 B12                        0.02
 B7                         0.017
 B9                         0.017
 B18                        0.016
 B5                         0.013
 B17                        0.013
 B8                         0.01
 B10                        0.009
 B6                         0.009


In [49]:
len(y_test)

204

# Optimize Model Performances(Hyperparameter)

In [50]:
s_time = time.time()

param_grid = {'n_estimators': [100,200,300], 'max_depth': [5,10,None]}
grid_search = GridSearchCV(model , param_grid= param_grid , cv = 5)
grid_search.fit(X_train , y_train)

best_model = grid_search.best_estimator_

e_time = time.time()
ex_time = e_time - s_time
print ('Execution Time:', ex_time, "seconds")

Execution Time: 23.16815447807312 seconds


# Apply Best model on Image

In [59]:
start_time = time.time()

#  Apply RF on image
image_reshaped = np.reshape(image, (image.shape[0], -1)).T
predicted_classes = best_model.predict(image_reshaped)
end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time/60, "Minute")


Execution time: 37.43722811539968 Minute


# Make predictions on the test set

In [60]:
s_time = time.time()

predicted_map = np.reshape(predicted_classes, (height,width))
y_predict = best_model.predict(X_test)
accuracy = accuracy_score(y_test , y_predict)
report = classification_report(y_test , y_predict)
print( "Model Accuracy:", accuracy)
print( "Classification Report :", report)

Model Accuracy: 0.8872549019607843
Classification Report :               precision    recall  f1-score   support

           4       1.00      1.00      1.00        66
           7       0.86      0.84      0.85        77
           8       0.81      0.82      0.81        61

    accuracy                           0.89       204
   macro avg       0.89      0.89      0.89       204
weighted avg       0.89      0.89      0.89       204



# Read G_truth vs predict_map

In [ ]:
classified_gt = pd.read_csv(r'gt.csv')
print(classified_gt.shape)
class_c = classified_gt['value'].value_counts()
print(class_c)
classified_gt.head(10)

In [ ]:
csv_gt = classified_gt
csv_gt = csv_gt.dropna()

gt_labels = csv_gt['value']
pre_labels = csv_gt['predicted']

accuracy  = accuracy_score(gt_labels, pre_labels)
print ('Accuracy:', accuracy)
report = classification_report(gt_labels, pre_labels)
print ('Classification Report:\n', report)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate confusion matrix
conf_matrix = confusion_matrix(gt_labels, pre_labels)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels= ['Agriculture','Water_body','manage_f','settlement','natural_f','rangeland','dense_shrub'], 
            yticklabels= ['Agriculture','Water_body','manage_f','settlement','natural_f','rangeland','dense_shrub'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix,precision_score, recall_score, f1_score

# Create a confusion matrix
confusion = confusion_matrix(gt_labels, pre_labels)

# Calculate commission, omission, and accuracy for each class
classes = [ 0,1, 2, 3, 4, 5]
for cls in classes:
    # Calculate true positives
    tp = confusion[cls, cls]

    # Calculate false positives
    fp = confusion[:, cls].sum() - tp

    # Calculate false negatives
    fn = confusion[cls, :].sum() - tp

    # Calculate true negatives
    tn = confusion.sum() - (tp + fp + fn)

    # Calculate commission
    commission = fp / (fp + tp)

    # Calculate omission
    omission = fn / (fn + tp)

    # Calculate accuracy
    accuracy = (tp + tn) / (tp + fp + fn + tn)

    print(f"Class {cls}:")
    print(f"Commission: {commission}")
    print(f"Omission: {omission}")
    print(f"Accuracy: {accuracy}")
    print()

In [ ]:
classes = [1,2, 3, 4, 5,6]

for cls in classes:
    # Calculate precision
    precision = precision_score(gt_labels, pre_labels, labels=[cls], average='micro')

    # Calculate recall
    recall = recall_score(gt_labels, pre_labels, labels=[cls], average='micro')

    # Calculate F1 score
    f1 = f1_score(gt_labels, pre_labels, labels=[cls], average='micro')

    print(f"Class {cls}:")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print()